In [ ]:
#%pip install qiskit
#%pip install qiskit-ionq
#%pip install qiskit-aer

import qiskit
import qiskit_ionq
from qiskit_ionq import IonQProvider

from qiskit import QuantumCircuit
from qiskit import transpile, assemble
from qiskit_ionq import IonQProvider

import math
import random
import numpy as np
import matplotlib.pyplot as plt

from scipy.optimize import minimize


# *********************** HIDE THIS BEFORE SHARING *********
provider = IonQProvider("")

#backend = Aer.get_backend('aer_simulator')
backend = provider.get_backend('ionq_qpu')
n_qubits = 3  # Number of system qubits.
acc = 3 # Number of digits of accuracy
n_shots = 10 ** (acc*2)  # Number of quantum measurements.
tot_qubits = n_qubits + 1  # Addition of an ancillary qubit.
ancilla_idx = n_qubits  # Index of the ancillary qubit (last position).
q_delta = 0.1  # Initial spread of random quantum weights

global runs
runs = 0

# q_delta = 0.1 => 31 iterations
# q_delta = 0.001 =>
eps = 0.01 # Required epsilon for minimizer thresholding

# Coefficients of the linear combination A = c_0 A_0 + c_1 A_1 ...
c = np.array([2.5, -1, -1, -0.5])

A_num = np.zeros((2**n_qubits, 2**n_qubits))
for idx in range(2**n_qubits):
    A_num[idx, idx] = 2

for m in range(2**n_qubits):
    for n in range(2**n_qubits):
        if abs(m - n) == 1:
            A_num[m, n] = -1

b = np.ones(2**n_qubits) / np.sqrt(2**n_qubits)

print("A = \n", np.real(A_num))
print("b = \n", b)
cond = np.linalg.cond(A_num)
print(str(cond))
A_inv = np.linalg.inv(A_num)
x = np.dot(A_inv, b)

c_probs = (x / np.linalg.norm(x)) ** 2

def U_b():
    global circ
    """Unitary matrix rotating the ground state to the problem vector |b> = U_b |0>."""
    for idx in range(n_qubits):
        circ.h(idx)

# Defines an C_i matrix, which will comprise an L2 matrix for the
# high-entanglement decomposition of the Poisson
def CI(i):
    global circ
    for j in range(i):
        circ.mcx([ancilla_idx, i], i-1-j, mode="noancilla")
    circ.mcx([ancilla_idx] + [x for x in range(i)], i, mode="noancilla")
    for j in range(i):
        circ.mcx([ancilla_idx, i], j, mode="noancilla")

# L3 tilda matrix for the high-entanglement decomposition of the poisson
# THIS ONE IS THE REASON THE RESULTS HAVE TO BE REVERSED.
# eventually we will want to invert this so we don't have to reverse the outputs
def L3():
    global circ
    circ.h(ancilla_idx)
    circ.mcx([x for x in range(n_qubits)], ancilla_idx, mode="noancilla")
    circ.h(ancilla_idx)

    for i in range(n_qubits):
        circ.cx(ancilla_idx, i)

    circ.h(ancilla_idx)
    circ.mcx([x for x in range(n_qubits)], ancilla_idx, mode="noancilla")
    circ.h(ancilla_idx)

    for i in range(n_qubits):
        circ.cx(ancilla_idx, i)


def CA(idx):
    global circ
    """Controlled versions of the unitary components A_l of the problem matrix A."""
    if idx == 0:
        # Identity operation
        None

    elif idx == 1:
        # L1
        circ.cx(ancilla_idx, 0)

    elif idx == 2:
        # L2 matrix
        for i in range(1, n_qubits):
            CI(i)

    elif idx == 3:
        # L3tilda matrix
        L3()


def variational_block(weights):
    global circ
    """Variational circuit mapping the ground state |0> to the ansatz state |x>."""
    num_layers = 3

    U_b()

    # Using num_layers layers for variational ansatz
    for layer in range(num_layers):
        # Layer of rotation gates
        for idx, element in enumerate(weights[layer*n_qubits:(layer+1)*n_qubits]):
            circ.ry(element, idx)

        # Applying entanglement layer to all but the last layer
        if layer < (num_layers-1):
            for idx in range(n_qubits):
                # Essentially vary the entanglement by the number
                # of layers we choose for the ansatz.
                if layer != idx:
                    circ.cz(layer, idx)

def had_test(weights, l, lp, j):
    global circ
    global runs

    print("Job: " + str(runs))
    print("Running...")
    circ = QuantumCircuit(n_qubits+1, 1)
    circ.h(ancilla_idx)

    # Initial state = V | 0 >
    variational_block(weights)

    # Unitary to control = A_lp_adj A_l
    CA(l)

    U_b()

    if j != -1:
        circ.cz(ancilla_idx, j)

    U_b()

    CA(lp)

    circ.h(ancilla_idx)

    circ.measure(ancilla_idx, 0)

    circ = transpile(circ, backend)
    result = backend.run(circ, shots=n_shots).result()

    output = result.get_counts(circ)
    o = (output['1'] / n_shots) if '1' in output.keys() else 0
    print("Completed.")
    runs += 1
    return (1 - 2*o)

def psi_psi(weights):

    res = 0

    # Add all l and lp s.t. l != lp and lp > l
    for l in range(len(c)):
        for lp in range(l+1, len(c)):
            res += 2 * c[l] * np.conj(c[lp]) * had_test(weights, l, lp, -1)

    # Add the remaining result of the summations
    for ind in range(len(c)):
        res += c[ind] * np.conj(c[ind])

    return abs(res)

def mu_sum(weights):

    res = 0

    for l in range(0, len(c)):
        for lp in range(l, len(c)):
            for j in range(n_qubits):
                if l == lp:
                    res += c[l] * np.conj(c[lp]) * had_test(weights, l, lp, j)
                else:
                    res += 2 * c[l] * np.conj(c[lp]) * had_test(weights, l, lp, j)
    return abs(res)

def cost_loc(weights):
    global cost, w
    w = weights

    mu_sum_val = mu_sum(weights)
    psi_psi_val = psi_psi(weights)

    cost = 0.5 - 0.5 * mu_sum_val / (n_qubits * psi_psi_val)
    return cost

In [ ]:
# TESTING MULTI-QUBIT SYSTEMS

from scipy.optimize import minimize

qubits_list = [10] # [x for x in range(3, 11)]
global w, circ

class GammaCondition(Exception):
    pass

def log_cost(x):
    global cost, cost_history
    cost_history.append(cost)
    print("Cost: " + str(cost))
    print("Weights: " + str(x))
    if cost <= gamma:
        raise GammaCondition

q_delta = 0.01
cost_history_all = []
for q in qubits_list:
    cost_history = []
    print(q)
    w = q_delta * np.random.rand(3*q)

    n_qubits = q  # Number of system qubits.
    tot_qubits = n_qubits + 1  # Addition of an ancillary qubit.
    ancilla_idx = n_qubits  # Index of the ancillary qubit (last position).
    circ = QuantumCircuit(tot_qubits, 1)
    A_num = np.zeros((2**n_qubits, 2**n_qubits))
    for idx in range(2**n_qubits):
        A_num[idx, idx] = 2

    for m in range(2**n_qubits):
        for n in range(2**n_qubits):
            if abs(m - n) == 1:
                A_num[m, n] = -1

    b = np.ones(2**q) / np.sqrt(2**q)

    print("A = \n", np.real(A_num))
    print("b = \n", b)
    cond = np.linalg.cond(A_num)
    print(str(cond))
    A_inv = np.linalg.inv(A_num)
    x = np.dot(A_inv, b)

    c_probs = (x / np.linalg.norm(x)) ** 2

    gamma = (eps ** 2) / (n_qubits * (cond ** 2))

    try:
        out = minimize(cost_loc, x0 = w, method='COBYLA', callback=log_cost)
    except GammaCondition:
        pass
    cost_history_all.append(cost_history)

10
A = 
 [[ 2. -1.  0. ...  0.  0.  0.]
 [-1.  2. -1. ...  0.  0.  0.]
 [ 0. -1.  2. ...  0.  0.  0.]
 ...
 [ 0.  0.  0. ...  2. -1.  0.]
 [ 0.  0.  0. ... -1.  2. -1.]
 [ 0.  0.  0. ...  0. -1.  2.]]
b = 
 [0.03125 0.03125 0.03125 ... 0.03125 0.03125 0.03125]
425801.6076001707
Job: 0
Running...
